In [1]:
import numpy as np
from tqdm.notebook import tqdm

# Step 0: Import our module!
import lbsimple as lb

In [2]:
# Step 1: Configure the simulation.
domain = lb.DomainMeta(0.002, lb.Sub(150, 150))
fluid = lb.FluidMeta(1000, 0.001, lb.Vect(0, 0))
sim = lb.Simulation(domain, fluid, 0.0001)

print(f"Speed of sound: {sim.c:.2f}")

Speed of sound: 20.00


In [3]:
def get_vortex(xx: np.array, yy: np.array, 
               x0: float, y0: float, 
               vx: float, vy: float, 
               vmax: float, 
               sigma: float, 
               clockwise: bool
               ):

    dxp = (xx - x0)
    dyp = (yy - y0)
    r = np.sqrt(dxp * dxp + dyp * dyp)
    v = (2 * r / (sigma * sigma)) * np.exp(-r * r / (sigma * sigma))
    theta = np.arctan2(dyp, dxp)

    sign = 1 if clockwise else -1
    vxout = +vmax * v * np.sin(theta) * sign + 2 * np.exp(-r * r / (sigma * sigma)) * vx
    vyout = -vmax * v * np.cos(theta) * sign + 2 * np.exp(-r * r / (sigma * sigma)) * vy

    return vxout, vyout

In [4]:
x = np.linspace(-15, 15, 150)
y = np.linspace(-15, 15, 150)
xx, yy = np.meshgrid(x, y)

vx =  np.zeros_like(xx)
vy = np.zeros_like(yy)

s_v = 0.15
dy_v = 2
dx_v = 10
for x_v, y_v, vx_v, c in [
    (-dx_v/2, dy_v/2, s_v, False),
    (-dx_v/2, -dy_v/2, s_v, True),
    (dx_v/2, dy_v/2, -s_v, True),
    (dx_v/2, -dy_v/2, -s_v, False),
]:
    vx_, vy_ = get_vortex(xx, yy, x_v, y_v, vx_v, 0, 0.3, 1, c)
    vx += vx_
    vy += vy_

vxy = np.stack([vx, vy], -1)

# vxy += (np.random.uniform(size=vxy.shape) - 0.5) * 0.001

# Step 2: Set the initial conditions.
sim.set_velocities(vxy)

In [5]:
# Step 3: Run the simulation and output the results to VTK files.
sim.write_grid_vtp(f"out/test_{0:06d}.vti")
for i in tqdm(list(range(1, 250))):
    # sim.run_for(0.1)
    sim.iterate_for(100)
    sim.write_grid_vtp(f"out/test_{i:06d}.vti")

  0%|          | 0/249 [00:00<?, ?it/s]